In [1]:
!pip install PyPDF2 openai numpy tiktoken requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.7 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

# Загружаем API-ключ из секретов Colab
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [3]:
import requests
from io import BytesIO
from PyPDF2 import PdfReader

# Список моделей и ссылок
documents = {
    "6095B": "https://agromester.md/pdf/6095B-New.pdf",
    "9R": "http://agromester.md/pdf/9r.pdf"
}

def extract_text_from_pdf(url):
    response = requests.get(url)
    response.raise_for_status()
    pdf_file = BytesIO(response.content)
    reader = PdfReader(pdf_file)
    text = ""
    for page in reader.pages:
        extracted = page.extract_text()
        if extracted:
            text += extracted + "\n"
    return text.strip()

# Загружаем тексты
model_texts = {}
for model, url in documents.items():
    print(f"Загрузка документа для модели {model}...")
    try:
        model_texts[model] = extract_text_from_pdf(url)
        print(f"Успешно загружено: {len(model_texts[model])} символов.")
    except Exception as e:
        print(f"Ошибка при загрузке {model}: {e}")
        model_texts[model] = ""

Загрузка документа для модели 6095B...
Успешно загружено: 29921 символов.
Загрузка документа для модели 9R...
Успешно загружено: 35132 символов.


In [4]:
import tiktoken

def split_into_chunks(text, max_tokens=400):
    tokenizer = tiktoken.get_encoding("cl100k_base")
    tokens = tokenizer.encode(text)
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk = tokenizer.decode(tokens[i:i + max_tokens])
        chunks.append(chunk)
    return chunks

# Храним чанки с меткой модели
knowledge_base = {}  # {model: [chunk1, chunk2, ...]}

for model, text in model_texts.items():
    if text:
        knowledge_base[model] = split_into_chunks(text)
        print(f"Модель {model}: разбито на {len(knowledge_base[model])} чанков.")
    else:
        knowledge_base[model] = []

Модель 6095B: разбито на 47 чанков.
Модель 9R: разбито на 41 чанков.


In [5]:
from openai import OpenAI
import numpy as np

client = OpenAI()

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

# Структура: {model: {"chunks": [...], "embeddings": np.array}}
vector_store = {}

for model, chunks in knowledge_base.items():
    if not chunks:
        vector_store[model] = {"chunks": [], "embeddings": np.array([])}
        continue

    print(f"Создание эмбеддингов для модели {model}...")
    embeddings = []
    for chunk in chunks:
        emb = get_embedding(chunk)
        embeddings.append(emb)

    vector_store[model] = {
        "chunks": chunks,
        "embeddings": np.array(embeddings)
    }
    print(f"✅ Эмбеддинги для {model} готовы.")

Создание эмбеддингов для модели 6095B...
✅ Эмбеддинги для 6095B готовы.
Создание эмбеддингов для модели 9R...
✅ Эмбеддинги для 9R готовы.


In [6]:
def detect_model_from_question(question):
    question_lower = question.lower()
    if "6095" in question_lower or "6095b" in question_lower:
        return "6095B"
    elif "9r" in question_lower:
        return "9R"
    else:
        # Если модель не указана — ищем в обеих
        return None

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def search_in_model(model, query, top_k=3):
    if model not in vector_store or len(vector_store[model]["embeddings"]) == 0:
        return []

    query_emb = get_embedding(query)
    embeddings = vector_store[model]["embeddings"]
    chunks = vector_store[model]["chunks"]

    scores = [cosine_similarity(query_emb, emb) for emb in embeddings]
    top_indices = np.argsort(scores)[-top_k:][::-1]
    return [chunks[i] for i in top_indices]

In [7]:
def ask_tractor_question(question):
    # Шаг 1: Определяем целевую модель
    target_model = detect_model_from_question(question)

    # Шаг 2: Ищем релевантные чанки
    relevant_chunks = []
    used_model = None

    if target_model:
        # Ищем только в указанной модели
        relevant_chunks = search_in_model(target_model, question, top_k=3)
        used_model = target_model
    else:
        # Ищем в обеих, выбираем модель с самым релевантным чанком
        best_score = -1
        best_chunk = ""
        best_model = None

        for model in vector_store:
            if not vector_store[model]["chunks"]:
                continue
            query_emb = get_embedding(question)
            embeddings = vector_store[model]["embeddings"]
            scores = [cosine_similarity(query_emb, emb) for emb in embeddings]
            max_score = max(scores)
            if max_score > best_score:
                best_score = max_score
                best_model = model
                best_idx = np.argmax(scores)
                best_chunk = vector_store[model]["chunks"][best_idx]

        if best_model and best_score > 0.5:  # порог релевантности
            relevant_chunks = [best_chunk]
            used_model = best_model
        else:
            return "К сожалению, по вашему вопросу нет информации в доступных спецификациях тракторов."

    # Шаг 3: Формируем контекст
    context = "\n\n".join(relevant_chunks)

    # Промпт: отвечаем как эксперт, без упоминания документов
    system_prompt = (
        "Вы — эксперт по сельскохозяйственной технике и консультант по тракторам. "
        "Отвечайте на вопросы покупателей чётко, точно и профессионально. "
        "НИКОГДА не упоминайте документы, PDF, спецификации или источники информации. "
        "Говорите так, будто вы знаете эти данные по умолчанию. "
        "Если информации нет — скажите: 'Эта информация не указана в технических данных трактора.'"
    )

    user_prompt = f"Информация по трактору: {context}\n\nВопрос клиента: {question}\n\nОтвет:"

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3,
        max_tokens=300
    )

    return response.choices[0].message.content.strip()

In [8]:
test_questions = [
    "Какая мощность двигателя у трактора 6095B?",
    "Какой объём топливного бака у 9R?",
    "Сколько весит трактор 6095B?",
    "Поддерживает ли 9R навесное оборудование ISOBUS?",
    "Какая максимальная скорость у трактора с индексом 9R?",
    "Какой расход топлива у трактора 6095B при полной нагрузке?"
]

for q in test_questions:
    print(f"❓ {q}")
    print(f"✅ {ask_tractor_question(q)}")
    print("-" * 80)

❓ Какая мощность двигателя у трактора 6095B?
✅ Номинальная мощность двигателя трактора 6095B составляет 95 л.с., а максимальная мощность — 96 л.с.
--------------------------------------------------------------------------------
❓ Какой объём топливного бака у 9R?
✅ Объем топливного бака у тракторов серии 9R/RT составляет 1325 литров.
--------------------------------------------------------------------------------
❓ Сколько весит трактор 6095B?
✅ Вес трактора 6095B составляет около 4200 кг без кабины и примерно 4000 кг с кабиной.
--------------------------------------------------------------------------------
❓ Поддерживает ли 9R навесное оборудование ISOBUS?
✅ Да, тракторы серии 9R поддерживают навесное оборудование ISOBUS, что обеспечивает экономию денежных средств и простоту управления орудиями.
--------------------------------------------------------------------------------
❓ Какая максимальная скорость у трактора с индексом 9R?
✅ Максимальная скорость трактора с индексом 9R составл